In [127]:

import os
from binance.client import Client
import pandas as pd
import datetime as dt
from backtesting import Backtest
from backtesting import Strategy
from backtesting.lib import crossover

In [128]:
# Binance API
api_key = os.environ.get('binance_api')
api_secret = os.environ.get('binance_secret')


In [129]:
client = Client(api_key, api_secret)

In [130]:
startdate = "14 Jun, 2022"
timeframe = "1h"
    
# coinPair = "BTCBUSD"

In [131]:
# client.get_account()

In [132]:
# from optparse import Values
def EMA(values, n):
    """
    Return exp moving average of `values`, at
    each step taking into account `n` previous values.
    """
    
    return pd.Series(values).ewm(span=n, adjust=False).mean()


In [133]:



# we will use four moving averages in total: 
# two moving averages whose relationship determines a general trend (we only trade long when the shorter MA is above the longer one, and vice versa), 
# and two moving averages whose cross-over with daily close prices determine the signal to enter or exit the position.
class EmaCross(Strategy):
    n1 = 10
    n2 = 64
    # n_enter = 20
    # n_exit = 10
    
    def init(self):
        # self.sma1 = self.I(EMA, self.data.Close, self.n1)
        # self.sma2 = self.I(EMA, self.data.Close, self.n2)
        
        self.ma1 = self.I(EMA, self.data.Close, self.n1)
        self.ma2 = self.I(EMA, self.data.Close, self.n2)
        
    def next(self):

        # If sma1 crosses above sma2, close any existing
        # short trades, and buy the asset
        if crossover(self.ma1, self.ma2):
            self.position.close()
            self.buy()

        # Else, if sma1 crosses below sma2, close any existing
        # long trades, and sell the asset
        elif crossover(self.ma2, self.ma1):
            self.position.close()
            self.sell()


In [134]:
def getdata(Symbol):
    frame = pd.DataFrame(client.get_historical_klines(Symbol,
                                                      timeframe,
                                                      # client.KLINE_INTERVAL_1HOUR,
                                                    #  '3 years ago UTC')
                                                      # '1 Feb, 2019 UTC', # bear market anterior
                                                      # '16 Nov, 2021 UTC' # inicio bear market 
                                                      # '14 Jun, 2022 UTC'   # 20k suporte
                                                      # '90 day ago UTC' 
                                                      startdate+' UTC'
                                                      # '4000 hour ago UTC' # 4hour
                                                      ))
    
    frame = frame.iloc[:,:6] # use the first 5 columns
    frame.columns = ['Time','Open','High','Low','Close','Volume'] #rename columns
    frame[['Open','High','Low','Close','Volume']] = frame[['Open','High','Low','Close','Volume']].astype(float) #cast to float
    # frame.Time = pd.to_datetime(frame.Time, unit='ms') #make human readable timestamp
    frame.index = [dt.datetime.fromtimestamp(x/1000.0) for x in frame.Time]
    return frame

In [135]:
def runBackTest(coinPair):

    print("coinPair=",coinPair)
    df = getdata(coinPair)
    df = df.drop(['Time'], axis=1)
    # df
    # df = bollinger_bands(df)


    bt = Backtest(df, EmaCross, cash=100000, commission=0.001)
    stats = bt.run()
    stats
    # bt.plot() 

    stats, heatmap = bt.optimize(
    n1=range(8, 50, 2),
    n2=range(10, 100, 2),
    constraint=lambda param: param.n1 < param.n2,
    maximize='Equity Final [$]',
    return_heatmap=True
    )

    # heatmap.dropna(inplace=True)
    # heatmap.droplevel
    # drop(labels inplace=True)
    # heatmap = heatmap[heatmap[0] > heatmap[1]]
    # from ast import Break
    # from doctest import BLANKLINE_MARKER


    dfbema = pd.DataFrame(heatmap.sort_values().iloc[-1:])
    n1 = dfbema.index.get_level_values(0)[0]
    n2 = dfbema.index.get_level_values(1)[0]
    returnPerc = round(stats['Return [%]'],2)

    # lista
    print("n1=",n1)
    print("n2=",n2)
    print("Return [%] = ",round(returnPerc,2))

    coinpairBestEma = pd.read_csv('coinpairBestEma')
    # coinpairBestEma
    # add to file coinpair Best Ema 
    # if exist then update else add
    linha = coinpairBestEma.index[(coinpairBestEma.coinPair == coinPair) & (coinpairBestEma.timeFrame == timeframe)].to_list()

    if not linha:
        print("There is no line with coinPair "+str(coinPair)+ " and timeframe "+str(timeframe)+". New line will be added.")
        #add linha
        coinpairBestEma.loc[len(coinpairBestEma.index)] = [coinPair, 
                                                            n1,
                                                            n2,
                                                            timeframe,
                                                            returnPerc
                                                        ]
    else:
        print("linha=",linha[0])
        # update linha
        coinpairBestEma.loc[linha[0],['slowEMA','fastEMA','returnPerc']] = [n1, n2, returnPerc]

    # coinpairBestEma
    print("Saving Coin Pair to csv")
    coinpairBestEma.to_csv('coinpairBestEma', index=False, header=True)


In [136]:
# heatmap.sort_values().iloc[-20:]

In [137]:

# stats._strategy

In [138]:
Listcoinpair = pd.read_csv('positioncheck')
# get coin pairs only
Listcoinpair = Listcoinpair.Currency
Listcoinpair

0        BTCBUSD
1        ETHBUSD
2        BNBBUSD
3        SOLBUSD
4      MATICBUSD
5        TWTBUSD
6        QNTBUSD
7       EGLDBUSD
8        BNXBUSD
9        UNIBUSD
10       CHZBUSD
11       LDOBUSD
12       ENSBUSD
13      NULSBUSD
14      KLAYBUSD
15      RNDRBUSD
16    SANTOSBUSD
17       VIBBUSD
18      REEFBUSD
19       FETBUSD
20      LINKBUSD
21     SUSHIBUSD
22      DYDXBUSD
23       GMXBUSD
24        OPBUSD
25     THETABUSD
Name: Currency, dtype: object

In [139]:
# run backtest for each coin pair
for coinPair in Listcoinpair:
    print("Backtest - Start")
    runBackTest(coinPair)

print("Backtest - End")

Backtest - Start
coinPair= BTCBUSD


/home/joaosilva/.local/lib/python3.10/site-packages/backtesting/_stats.py:122: RuntimeWarning: divide by zero encountered in double_scalars
  s.loc['Sortino Ratio'] = np.clip((annualized_return - risk_free_rate) / (np.sqrt(np.mean(day_returns.clip(-np.inf, 0)**2)) * np.sqrt(annual_trading_days)), 0, np.inf)  # noqa: E501
/home/joaosilva/.local/lib/python3.10/site-packages/backtesting/backtesting.py:1488: UserWarning: Searching for best of 735 configurations.
  output = _optimize_grid()
/home/joaosilva/.local/lib/python3.10/site-packages/backtesting/_stats.py:122: RuntimeWarning: divide by zero encountered in double_scalars
  s.loc['Sortino Ratio'] = np.clip((annualized_return - risk_free_rate) / (np.sqrt(np.mean(day_returns.clip(-np.inf, 0)**2)) * np.sqrt(annual_trading_days)), 0, np.inf)  # noqa: E501
/home/joaosilva/.local/lib/python3.10/site-packages/backtesting/_stats.py:122: RuntimeWarning: divide by zero encountered in double_scalars
  s.loc['Sortino Ratio'] = np.clip((annualized

n1= 8
n2= 52
Return [%] =  1.08
linha= 0
Saving Coin Pair to csv
Backtest - Start
coinPair= ETHBUSD


/home/joaosilva/.local/lib/python3.10/site-packages/backtesting/backtesting.py:1488: UserWarning: Searching for best of 735 configurations.
  output = _optimize_grid()


n1= 8
n2= 18
Return [%] =  2.56
There is no line with coinPair ETHBUSD and timeframe 1h. New line will be added.
Saving Coin Pair to csv
Backtest - Start
coinPair= BNBBUSD


...

n1= 48
n2= 98
Return [%] =  2.6
There is no line with coinPair GMXBUSD and timeframe 1h. New line will be added.
Saving Coin Pair to csv
Backtest - Start
coinPair= OPBUSD


/home/joaosilva/.local/lib/python3.10/site-packages/backtesting/backtesting.py:1488: UserWarning: Searching for best of 735 configurations.
  output = _optimize_grid()


n1= 8
n2= 10
Return [%] =  3.39
There is no line with coinPair OPBUSD and timeframe 1h. New line will be added.
Saving Coin Pair to csv
Backtest - Start
coinPair= THETABUSD


/home/joaosilva/.local/lib/python3.10/site-packages/backtesting/backtesting.py:1488: UserWarning: Searching for best of 735 configurations.
  output = _optimize_grid()
/home/joaosilva/.local/lib/python3.10/site-packages/backtesting/_stats.py:122: RuntimeWarning: divide by zero encountered in double_scalars
  s.loc['Sortino Ratio'] = np.clip((annualized_return - risk_free_rate) / (np.sqrt(np.mean(day_returns.clip(-np.inf, 0)**2)) * np.sqrt(annual_trading_days)), 0, np.inf)  # noqa: E501
/home/joaosilva/.local/lib/python3.10/site-packages/backtesting/_stats.py:122: RuntimeWarning: divide by zero encountered in double_scalars
  s.loc['Sortino Ratio'] = np.clip((annualized_return - risk_free_rate) / (np.sqrt(np.mean(day_returns.clip(-np.inf, 0)**2)) * np.sqrt(annual_trading_days)), 0, np.inf)  # noqa: E501
/home/joaosilva/.local/lib/python3.10/site-packages/backtesting/_stats.py:122: RuntimeWarning: divide by zero encountered in double_scalars
  s.loc['Sortino Ratio'] = np.clip((annualized

n1= 8
n2= 10
Return [%] =  5.41
There is no line with coinPair THETABUSD and timeframe 1h. New line will be added.
Saving Coin Pair to csv
Backtest - End
